In [22]:
import numpy as np
import pandas as pd
from glob import glob
import os
import cv2
import xml.etree.ElementTree as xet


In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images only\Cars0.xml,226,419,125,173
1,./images only\Cars1.xml,134,262,128,160
2,./images only\Cars10.xml,140,303,5,148
3,./images only\Cars100.xml,175,214,114,131
4,./images only\Cars101.xml,167,240,202,220


# Labels For Yolo
    center_x , center_y , width , height

In [3]:
# Parsing the data

def parsing(path):
    
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'./images/{name}'
    
    # Width and Height
    
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)

In [4]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,./images only\Cars0.xml,226,419,125,173,./images/Cars0.png,500,268
1,./images only\Cars1.xml,134,262,128,160,./images/Cars1.png,400,248
2,./images only\Cars10.xml,140,303,5,148,./images/Cars10.png,400,225
3,./images only\Cars100.xml,175,214,114,131,./images/Cars100.png,400,267
4,./images only\Cars101.xml,167,240,202,220,./images/Cars101.png,400,300


In [5]:
# Center_x,center_y,width,height

df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [6]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images only\Cars0.xml,226,419,125,173,./images/Cars0.png,500,268,0.64500,0.555970,0.3860,0.179104
1,./images only\Cars1.xml,134,262,128,160,./images/Cars1.png,400,248,0.49500,0.580645,0.3200,0.129032
2,./images only\Cars10.xml,140,303,5,148,./images/Cars10.png,400,225,0.55375,0.340000,0.4075,0.635556
3,./images only\Cars100.xml,175,214,114,131,./images/Cars100.png,400,267,0.48625,0.458801,0.0975,0.063670
4,./images only\Cars101.xml,167,240,202,220,./images/Cars101.png,400,300,0.50875,0.703333,0.1825,0.060000


In [16]:
df_train = df.iloc[:375]
df_test = df.iloc[375:]

In [17]:
from shutil import copy

In [20]:
train_folder = './Data_images/train'
values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values

for fname,x,y,w,h in values :
    
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # Copy  each image into the folder
    copy(fname,dst_image_path)
    
    # Generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h} '
    with open(dst_label_file,'w') as f:
        f.write(label_txt)
        
        f.close()

In [21]:
test_folder = './Data_images/test'
values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values

for fname,x,y,w,h in values :
    
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # Copy  each image into the folder
    copy(fname,dst_image_path)
    
    # Generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h} '
    with open(dst_label_file,'w') as f:
        f.write(label_txt)
        
        f.close()